# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [ ]:
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [ ]:
orders = pd.read_csv('Orders.csv')

orders.head()

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# Sub Problem 1

# Group by 'CustomerID' and aggregate (sum) 'amount_spent'
customers_spent = orders.groupby('CustomerID')['amount_spent'].agg('sum').reset_index()

customers_spent.head()

In [ ]:
# Sub Problem 2

# Select customer groups 'VIP', 'Preferred' by 'amount_spent' quantile range
customers_vip = customers_spent[customers_spent['amount_spent'] > customers_spent['amount_spent'].quantile(0.95)]
customers_pref = customers_spent[(customers_spent['amount_spent'] >= customers_spent['amount_spent'].quantile(0.75))
                                 & (customers_spent['amount_spent'] <= customers_spent['amount_spent'].quantile(0.95))]

display(customers_vip.sort_values(by = 'amount_spent'))
display(customers_pref.sort_values(by = 'amount_spent'))

In [ ]:
# Sub Problem 3

# Label selected customers as 'VIP' or 'Preferred'
customers_spent['CustomerLabel'] = pd.qcut(customers_spent['amount_spent'], [0, 0.75, 0.95, 1], labels = ['', 'Preferred', 'VIP'])

customers_spent.head()

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# Joining 'Country' to VIP customers
country_vip = pd.merge(left = customers_vip,
                       right = orders[['CustomerID', 'Country']],
                       how = 'left',
                       left_on = 'CustomerID',
                       right_on = 'CustomerID')

country_vip.drop_duplicates('CustomerID', inplace = True)

country_vip['Country'].value_counts().head(1)

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# Concatenating VIP + Preferred customers dfs
customers_vip_pref = pd.concat([customers_vip, customers_pref])

# Joining 'Country' to VIP/Preferred customers
country_vip_pref = pd.merge(left = customers_vip_pref,
                       right = orders[['CustomerID', 'Country']],
                       how = 'left',
                       left_on = 'CustomerID',
                       right_on = 'CustomerID')

country_vip_pref.drop_duplicates('CustomerID', inplace = True)

country_vip_pref['Country'].value_counts().head(1)